In [2]:
#
# Google Drive に pota フォルダーと pota/maps フォルダーを作成
# Japan (JA).csv：Map List の Japan を選択し、CSファイルをダウンロード。potaフォルダーにコピー
#

# ライブラリのインポート
import os
import shutil
import folium
import pandas as pd

# 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige',
# 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
# 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'

# none, hunt, activate, hunt/activate
point_colors = ['red', 'blue', 'green', 'orange']

print("Begin")

# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

# pota/maps ディレクトリの削除
shutil.rmtree("/content/drive/My Drive/pota/maps")
# pota/mas ディレクトリの作成
os.mkdir("/content/drive/My Drive/pota/maps")

# アクティベート、ハント情報の読み込み
acthunt=pd.read_csv("/content/drive/My Drive/pota/Japan (JA).csv")
acthunt = acthunt.rename(columns={"reference":"Reference"})

# 都道府県情報の読み込み
prefectures = pd.read_json("https://tyjcode.github.io/y001/prefectures.json")

# アクティベート、ハント情報に都道府県情報をマージ
acthunt_pref = pd.merge(acthunt, prefectures, on="locationDesc")

# 都道府県でグルーピング
acthunt_pref_group = acthunt_pref.groupby("locationName").sum().iloc[:,:7].reset_index()

# 公園情報の読み込み
parks = pd.read_json("https://tyjcode.github.io/y001/parks-latest.json")
parks = parks.drop({"latitude","longitude"}, axis=1)

# 公園情報をマージ
acthunt_pref_map = pd.merge(acthunt_pref, parks, on="Reference")
acthunt_pref_map.loc[acthunt_pref_map["my_activations"] >= 0,"color"] = (acthunt_pref_map["my_activations"] > 0 )* 2 + (acthunt_pref_map["my_hunted_qsos"]> 0)
acthunt_pref_map.head()


#
# 全国地図にプロット
# max_widthを入れないと縦書きになる
#
map = folium.Map(location=[35.7503, 139.523], zoom_start=6)

for i, r in acthunt_pref_map.iterrows():
  name_pop = folium.Popup(f'{r["Reference"]}<br/>{r["日本語名"]}<br/>{r["my_activations"]}  {r["my_hunted_qsos"]}', max_width=1000)
  icon_color = point_colors[r["color"]]
  folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop,icon=folium.Icon(color=icon_color)).add_to(map)

# 地図の作成
map.save("/content/drive/My Drive/pota/maps/map_parks.html")

#
# ハントした都道府県別に地図を作成
#
for j, h in acthunt_pref_group.iterrows():
  #print(h["locationName"],h["my_activations"],h["my_hunted_qsos"])

  # 都道府県毎に抽出
  acthunt_pref_map_pref = acthunt_pref_map.query('locationName_x=="{}"'.format(h["locationName"])).reset_index()

  # ２つ目の公園を中心に地図を作成
  map2 = folium.Map(location=[acthunt_pref_map_pref.loc[1,"latitude"], acthunt_pref_map_pref.loc[1,"longitude"]], zoom_start=10)

  # 公園をポイント
  for i, r in acthunt_pref_map_pref.iterrows():
    name_pop = folium.Popup(f'{r["Reference"]}<br/>{r["日本語名"]}<br/>{r["my_activations"]}  {r["my_hunted_qsos"]}', max_width=1000)
    icon_color = point_colors[r["color"]]
    folium.Marker(location=[r["latitude"], r["longitude"]], popup=name_pop,icon=folium.Icon(color=icon_color)).add_to(map2)

  # 地図の作成
  map2.save(f'/content/drive/My Drive/pota/maps/map_parks2_{h["locationName"]}_{h["my_activations"]}_{h["my_hunted_qsos"]}.html')

print("End")

Begin
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-2-ebc4c3592e27>:41: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acthunt_pref_group = acthunt_pref.groupby("locationName").sum().iloc[:,:7].reset_index()
<ipython-input-2-ebc4c3592e27>:49: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  acthunt_pref_map.loc[acthunt_pref_map["my_activations"] >= 0,"color"] = (acthunt_pref_map["my_activations"] > 0 )* 2 + (acthunt_pref_map["my_hunted_qsos"]> 0)


End
